In [1]:
import torch

In [3]:
print(torch.cuda.is_available())

False


In [28]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [4]:
torch.manual_seed(42)

In [6]:
df_train = pd.read_csv('fashion-mnist_train.csv')
df_test = pd.read_csv('fashion-mnist_test.csv')

In [7]:
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_train.shape

(60000, 785)

In [20]:
X_train = df_train.iloc[:,1:].values
y_train = df_train.iloc[:,0].values

In [21]:
df_test.shape

(10000, 785)

In [23]:
X_test = df_test.iloc[:,1:].values
y_test = df_test.iloc[:,0].values

In [32]:
#Rescaling the pixel value

X_train = X_train/ 255.0
X_test = X_test/255.0

In [43]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype = torch.float32) # features must be in float32
        self.labels = torch.tensor(labels, dtype = torch.long) # labels must be in torch.long
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    
        

In [44]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [45]:
train_dataloader = DataLoader(train_dataset,
                              shuffle=True,
                              batch_size=32)
test_dataloader = DataLoader(test_dataset,
                             shuffle=False,
                             batch_size=32)

In [50]:
class ANN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features , 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,10),
        )
        
    def forward(self, x):
        return self.model(x)
    

In [93]:
EPOCHS = 100

LEARNING_RATE = 0.1

model = ANN(X_train.shape[1])

#loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), LEARNING_RATE)

In [94]:
#training loop

for epoch in range(EPOCHS):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_dataloader:
        #forward pass
        outputs = model(batch_features)
        
        
        #loss calculate
        loss = loss_fn(outputs, batch_labels)
        
        #back pass
        optimizer.zero_grad()
        loss.backward()
        
        #update grads
        optimizer.step()
        
        total_epoch_loss = total_epoch_loss + loss.item()
    
    avg_loss = total_epoch_loss / len(train_dataloader)
    print(f"Epoch: {epoch + 1}, Loss: {avg_loss}")
    

Epoch: 1, Loss: 0.5906458683649699
Epoch: 2, Loss: 0.4038810153325399
Epoch: 3, Loss: 0.36227092726627985
Epoch: 4, Loss: 0.33563381238778434
Epoch: 5, Loss: 0.31416683295170467
Epoch: 6, Loss: 0.2985961933165789
Epoch: 7, Loss: 0.28569419141908486
Epoch: 8, Loss: 0.2730976735095183
Epoch: 9, Loss: 0.2627592338939508
Epoch: 10, Loss: 0.2531068742106358
Epoch: 11, Loss: 0.24324719735185305
Epoch: 12, Loss: 0.2352496812661489
Epoch: 13, Loss: 0.22866382907629013
Epoch: 14, Loss: 0.22170659115215144
Epoch: 15, Loss: 0.2174235051472982
Epoch: 16, Loss: 0.21049945144702992
Epoch: 17, Loss: 0.20577406233151754
Epoch: 18, Loss: 0.1979283615514636
Epoch: 19, Loss: 0.19222046816001337
Epoch: 20, Loss: 0.18589118901491164
Epoch: 21, Loss: 0.18244263659318288
Epoch: 22, Loss: 0.17951483222420017
Epoch: 23, Loss: 0.17303123316044608
Epoch: 24, Loss: 0.16843880413944523
Epoch: 25, Loss: 0.16908084724197786
Epoch: 26, Loss: 0.1607700129946073
Epoch: 27, Loss: 0.15489607999597987
Epoch: 28, Loss: 0.1

In [95]:
model.eval()

ANN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [96]:
#evaluation code

total = 0
correct = 0


with torch.no_grad():
    for batch_features, batch_labels in test_dataloader:
        
        outputs = model(batch_features) # shape of outputs will be (32,10) like each data will have 10 prob.
        predicted = outputs.argmax(dim=1)
        
    total = total + batch_labels.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()
print(f"Testing accuarcy is {correct*100/total}" )
        

Testing accuarcy is 93.75


In [98]:

#evaluation code

total = 0
correct = 0


with torch.no_grad():
    for batch_features, batch_labels in train_dataloader:
        
        outputs = model(batch_features) # shape of outputs will be (32,10) like each data will have 10 prob.
        predicted = outputs.argmax(dim=1)
        
    total = total + batch_labels.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()
print(f"Training accuarcy is {correct*100/total}" )
        

Training accuarcy is 100.0


### Model is overfitting..gonna sleep..will see after waking up

as `training accuracy is 100.0` and `testing accuracy is 93.75`